In [ ]:
import networkx as nx
import pandas as pd
from functools import reduce
from swiplserver import PrologMQI, PrologThread
import sys
import os
import glob
from itertools import chain, combinations

In [ ]:
G = nx.DiGraph(nx.nx_pydot.read_dot("graph_complete.dot"))

In [ ]:
class Target:
    def __init__(self,pr,ab):
        self.present = pr
        self.absent = ab
    
    def __str__(self):
        return f"present: {self.present} absent: {self.absent}"    
        
targets = []
interesting_genes = ["tbet","gata3","foxp3","rorgt"]
def powerset(s):
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))
for x in powerset(interesting_genes):
    if (len(x)>0):
        targets.append(Target([k for k in x],[k for k in interesting_genes if k not in x]))

In [ ]:
# checks whether a node is in an attractor
def check_node(node):
    cycle = list(nx.find_cycle(G,node))
    tmp = map(lambda x : x[0]==node or x[1]==node, cycle) 
    return reduce(lambda b1, b2: b1 or b2, tmp)

# compute the list of genes that are present in the attractor reachable form "node"
def compute_attractor(node):
    cycle = list(nx.find_cycle(G,node))
    tmp1 = map(lambda x: x[0].split(';') + x[1].split(';'), cycle)
    tmp2 = reduce(lambda x, y: x+y,tmp1)
    res = list(dict.fromkeys(tmp2))
    return res

In [ ]:
def count_states (d):
    tmp = 0
    for v in d.values():
        tmp += len(v)
    return tmp

# finds computations (LTS traces) that lead to the "target"
def target_computations(target):
    all_nodes = list(G.nodes)
    filtered = [k for k in all_nodes if check_node(k)] # filters out intermediate nodes (not in attractor)
    attractors_map = dict()
    for f in filtered:
        attractors_map[f] = compute_attractor(f) # creates a map "state -> attractor"
    for s in target.present:
        filtered = [k for k in filtered if s in attractors_map[k]]   # filters out states that do not contain s (present in the target)
    for s in target.absent:
        filtered = [k for k in filtered if s not in attractors_map[k]] # filters out state that contain s (absent in the target)

    # filters out states in target attractors, but that do not contain any gene in target.present
    # (slicing would give an empty result on these states)
    filtered2 = [k for k in filtered if len(list(set(k.split(';')) & set(target.present)))>0]

    # cleans the attractors_map from states not in target (this step is not really necessary...)
    keys_to_delete = list() 
    for k in attractors_map.keys():
        if k not in filtered: keys_to_delete.append(k) 
    for k in keys_to_delete:
        del attractors_map[k] 
    
    contexts = list()   # list of the contexts that lead to the target
    contexts_dict = {}  # for each context, lists the states in the corresponding attractor
    filtered_splitted = [f.split(';') for f in filtered2]
    for f in filtered_splitted:
        prefix = f[0:9] # the first 9 elements in the state are the context
        pure_state = f[9:] # the others are the actual state
        if (not prefix in contexts):
            contexts.append(prefix)
            contexts_dict[','.join(prefix)] = [','.join(pure_state)]
        else:
            contexts_dict[','.join(prefix)].append(','.join(pure_state))
    print("TARGET --> " + str(target))
    print("found " + str(len(contexts)) + " contexts that lead to the target")
    print("found " + str(count_states(contexts_dict) + (len(filtered)-len(filtered2))) + " states in reachable attractors")
    print("of which " + str(count_states(contexts_dict)) + " with genes in the target")
    print()
    return contexts, contexts_dict

In [ ]:
target_tot = len(targets)
target_count = 0
print("TO BE ANALYZED: " + str(target_tot) + " target")
print()

outfile = open("out.txt","w")
for target in targets:
    target_count = target_count+1
    print("TARGET COUNT: " + str(target_count) + "/" + str(target_tot))
    contexts, contexts_dict = target_computations(target)

    prolog_target = ','.join(target.present)

    cont = 1
    tot = str(count_states(contexts_dict))
    union_set = set()
    intersection_set = set()
    first_time = True
    for ctx in contexts:
        prolog_context = ','.join(ctx)
        prolog_target_states = contexts_dict[','.join(ctx)]
        for i,state in enumerate(prolog_target_states):
            print("TEST CASE: " + str(cont) + "/" + str(tot))
            cont=cont+1
            print("CONTEXT: " + prolog_context + "      ATTRACTOR STATE: " + str(i+1) + "/" + str(len(prolog_target_states)))
            print("STATE: " + state)
            param_file = open("Bioresolve_depthfirst/params.pl",'w')
            param_file.write("myenvironment('[ x1 = ({tgfb}.x11 + {}.x0),\n x2 = ({il23}.x21 + {}.x0),\n x3 = ({il12}.x31 + {}.x0),\n x4 = ({il18}.x41 + {}.x0),\n x5 = ({il4e}.x51 + {}.x0),\n x6 = ({il27}.x61 + {}.x0),\n x7 = ({il6e}.x71 + {}.x0),\n x8 = ({ifnge}.x81 + {}.x0),\n x9 = ({tcr}.x91 + {}.x0),\n x11 = {tgfb}.x11,\n x21 = {il23}.x21,\n x31 = {il12}.x31,\n x41 = {il18}.x41,\n x51 = {il4e}.x51,\n x61 = {il27}.x61,\n x71 = {il6e}.x71,\n x81 = {ifnge}.x81,\n x91 = {tcr}.x91,\n x0 = {}.x0\n]').\n \nmyentities([]).\n \n")
            param_file.write('mycontext("[' + prolog_context + ']").\n\n')
            param_file.write('mymonitor("[ m0 ]").\n\n')
            param_file.write('mymondef("[ m0 = ([{' + state + '} inW].no({' + prolog_target +'}) + [-({' + state + '} inW)].m0) ]").\n')
            param_file.flush()
            param_file.close()
            print()
            with PrologMQI() as mqi:
                with mqi.create_thread() as prolog_thread:
                    prolog_thread.query('["BioResolve_depthfirst/filterBioResolve.pl"]')
                    result = prolog_thread.query('main_do(slice,S).')
                    tmp_set = set(chain.from_iterable(result[0]['S'])) 
                    union_set.update(tmp_set)
                    if (first_time):
                        intersection_set = tmp_set
                        first_time = False
                    else:
                        intersection_set = intersection_set.intersection(tmp_set)
    for f in glob.glob("tmp-slice*.txt"):
        os.remove(f)
        
    print()
    union_set=sorted(union_set)
    intersection_set=sorted(intersection_set)
    print("SET OF ENTITIES IN SLICED COMPUTATIONS FOR TARGET " + str(target) + ":")
    outfile.write("SET OF ENTITIES IN SLICED COMPUTATIONS FOR TARGET " + str(target) + ":\n")
    print("UNION: " + str(union_set))
    outfile.write("         UNION: " + str(union_set)+"\n")
    print("INTERSECTION: " + str(union_set))
    outfile.write("         INTERSECTION: " + str(intersection_set)+"\n\n")
    print()
    outfile.flush()

outfile.close()

In [ ]:
for target in targets:
    contexts, contexts_dict = target_computations(target)
    if (len(contexts)>0):
        df = pd.DataFrame(contexts).sort_values(by=[8,7,6,5,4,3,2,1,0],ignore_index=True)
        print("CONTEXTS THAT LEAD TO THE TARGET:")
        print(df)
        print()
        df.to_csv("contexts_to_" + str(target.present) + ".csv",index=False,header=False)